In [12]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
 #   transforms.Resize(256),             # they are already resized in processing
    transforms.CenterCrop(227),       # AlexNet input size is 227
    transforms.ToTensor(),             
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

dataset = datasets.ImageFolder('color', transform=transform)

train_size = int(0.8 * len(dataset))
validation_size = len(dataset) - train_size
train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)


54305


In [13]:
from torchvision.models import AlexNet_Weights
from torchvision import models
#pretrained alexnet model to test
model = models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)

model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 38)


In [14]:
import torch.optim as optim

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()

for i, (inputs, labels) in enumerate(train_loader):
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    if (i + 1) % 10 == 0:
        print(f'Batch {i + 1}, Loss: {loss.item()}')

torch.save(model.state_dict(), 'alexnet_plantvillage.pth')
print('Training complete and model saved')

Batch 10, Loss: 3.588106155395508
Batch 20, Loss: 3.5481505393981934
Batch 30, Loss: 3.5763235092163086
Batch 40, Loss: 3.259951114654541
Batch 50, Loss: 3.231973648071289
Batch 60, Loss: 3.427610158920288
Batch 70, Loss: 3.315734386444092
Batch 80, Loss: 3.3667454719543457
Batch 90, Loss: 3.4248456954956055
Batch 100, Loss: 3.3846139907836914
Batch 110, Loss: 3.2260706424713135
Batch 120, Loss: 3.2688100337982178
Batch 130, Loss: 3.332181453704834
Batch 140, Loss: 3.2506771087646484
Batch 150, Loss: 3.6070151329040527
Batch 160, Loss: 3.3267838954925537
Batch 170, Loss: 3.171787977218628
Batch 180, Loss: 3.3601369857788086
Batch 190, Loss: 3.4935195446014404
Batch 200, Loss: 3.331143379211426
Batch 210, Loss: 3.1557068824768066
Batch 220, Loss: 3.1612601280212402
Batch 230, Loss: 3.4984817504882812
Batch 240, Loss: 3.3776192665100098
Batch 250, Loss: 3.494133234024048
Batch 260, Loss: 3.17972993850708
Batch 270, Loss: 3.3125667572021484
Batch 280, Loss: 3.493208885192871
Batch 290, Lo

In [16]:
def evaluate_model():
    model.eval()  # Ensure the model is in evaluation mode.
    model.cuda()  # Move model to GPU.
    
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient computation for inference.
        for inputs, labels in validation_loader:
            inputs, labels = inputs.cuda(), labels.cuda()  # Move inputs and labels to GPU.
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy}%')

evaluate_model()

Accuracy: 10.30291869993555%
